In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110510

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,70634.1233
2018-02-28,57874.5137
2018-03-31,61273.9846
2018-04-30,67588.1236
2018-05-31,68677.0149
2018-06-30,65347.5650
2018-07-31,67802.8579
2018-08-31,58425.6064
2018-09-30,63319.8498


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    70634.1233
 2018-02-28    57874.5137
 2018-03-31    61273.9846
 2018-04-30    67588.1236
 2018-05-31    68677.0149
 2018-06-30    65347.5650
 2018-07-31    67802.8579
 2018-08-31    58425.6064
 2018-09-30    63319.8498
 2018-10-31    63453.0668
 2018-11-30    52083.3285
 2018-12-31    63039.4631
 2019-01-31    52524.3699
 2019-02-28    68447.3350
 2019-03-31    62600.4430
 2019-04-30    68879.5754
 2019-05-31    65264.4997
 2019-06-30    80363.0617
 2019-07-31    91809.7364
 2019-08-31    73141.9668
 2019-09-30    72649.8491
 2019-10-31    79121.9361
 2019-11-30    78772.0954
 2019-12-31    81712.3830
 2020-01-31    93203.0810
 2020-02-29    31014.2270
 2020-03-31    60883.5858
 2020-04-30    58560.4480
 2020-05-31    52671.5398
 2020-06-30    67440.5552
 2020-07-31    67657.2210
 2020-08-31    52796.6725
 2020-09-30    50362.0720
 2020-10-31    65742.8595
 2020-11-30    57955.1902
 2020-12-31    58122.7843
 2021-01-31    57015.7638
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.924740
p-value : 0.000031
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=904.304, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=904.803, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=906.632, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=907.423, Time=0.04 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=910.870, Time=0.09 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=910.931, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=903.939, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=905.058, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=906.945, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=909.195, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=912.136, Time=0.11 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=904.760, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=906.444, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=907.856, Time=0.26 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=909.814, Time=0.42 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -448.965
Method:                       css-mle   S.D. of innovations          10615.307
Date:                Sat, 11 Sep 2021   AIC                            903.931
Time:                        03:59:09   BIC                            909.144
Sample:                             0   HQIC                           905.841
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.495e+04   2118.773     30.657      0.000    6.08e+04    6.91e+04
ar.L1.y        0.2323      0.148      1.565      0.118      -0.059       0.523
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([65233.2914714 , 65019.20881061]), array([10615.30672702, 10898.03768982]), array([[44427.67260159, 86038.91034122],
       [43659.4474364 , 86378.97018481]]))
